# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [6]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)
#rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)
preprocessed.rename(columns = {'1stFlrSF_log':'FirstFlrSF_log'},inplace=True)

## Perform stepwise selection

The function for stepwise selection is copied below. Use this provided function on your preprocessed Ames Housing data.

In [7]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [8]:
# Your code here
result = stepwise_selection(preprocessed.drop('SalePrice_log', axis=1), preprocessed['SalePrice_log'], verbose=True)
print('resulting features:')
print(result)

<ipython-input-7-49c095109920>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-7-49c095109920>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Add  GrLivArea_log                  with p-value 1.59847e-243
Add  KitchenQual_TA                 with p-value 1.56401e-67


<ipython-input-7-49c095109920>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-7-49c095109920>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Add  FirstFlrSF_log                 with p-value 7.00069e-48
Add  KitchenQual_Fa                 with p-value 1.70471e-37


<ipython-input-7-49c095109920>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-7-49c095109920>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


...

<ipython-input-7-49c095109920>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Add  MSZoning_RH                    with p-value 3.95797e-05


<ipython-input-7-49c095109920>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Add  Neighborhood_NWAmes            with p-value 0.00346099


ValueError: list.remove(x): x not in list

### Build the final model again in Statsmodels

In [9]:
# Your code here
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

from statsmodels.formula.api import ols
#predictors

outcome = 'SalePrice_log'
predictors = preprocessed.drop('SalePrice_log', axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

#model fit
model = ols(formula=formula, data=preprocessed).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     156.5
Date:                Wed, 11 Aug 2021   Prob (F-statistic):               0.00
Time:                        02:24:32   Log-Likelihood:                -738.14
No. Observations:                1460   AIC:                             1572.
Df Residuals:                    1412   BIC:                             1826.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.1317      0.263     -0.500      0.617      -0.648       0.385
LotArea_log              0.1033      0.019      5.475      0.000       0.066       0.140
FirstFlrSF_log           0.1371      0.016      8.584      0.000       0.106       0.168
GrLivArea_log            0.3768      0.016     24.114      0.000       0.346       0.407
BldgType_2fmCon         -0.1715      0.079     -2.173      0.030      -0.326      -0.017
BldgType_Duplex         -0.4203      0.062     -6.813      0.000      -0.541      -0.299
BldgType_Twnhs          -0.1403      0.093     -1.513      0.130      -0.322       0.042
BldgType_TwnhsE         -0.0512      0.060     -0.858      0.391      -0.168       0.066
KitchenQual_Fa          -0.9999      0.088    -11.315      0.000      -1.173      -0.827
KitchenQual_Gd          -0.3820      0.050     -7.613      0.000      -0.480      -0.284
KitchenQual_TA          -0.6692      0.055    -12.111      0.000      -0.778      -0.561
SaleType_CWD             0.2285      0.215      1.061      0.289      -0.194       0.651
SaleType_Con             0.5861      0.304      1.927      0.054      -0.010       1.183
SaleType_ConLD           0.3151      0.155      2.029      0.043       0.010       0.620
SaleType_ConLI           0.0331      0.195      0.169      0.865      -0.350       0.416
SaleType_ConLw           0.0161      0.196      0.082      0.935      -0.368       0.400
SaleType_New             0.2999      0.079      3.803      0.000       0.145       0.455
SaleType_Oth             0.1178      0.245      0.480      0.631      -0.364       0.599
SaleType_WD              0.1748      0.065      2.676      0.008       0.047       0.303
MSZoning_FV              1.0666      0.193      5.526      0.000       0.688       1.445
MSZoning_RH              0.8768      0.194      4.512      0.000       0.496       1.258
MSZoning_RL              0.9961      0.162      6.151      0.000       0.678       1.314
MSZoning_RM              1.1023      0.152      7.264      0.000       0.805       1.400
Street_Pave             -0.2131      0.180     -1.182      0.237      -0.567       0.141
Neighborhood_Blueste     0.0529      0.318      0.167      0.868      -0.571       0.677
Neighborhood_BrDale     -0.4627      0.171     -2.711      0.007      -0.798      -0.128
Neighborhood_BrkSide    -0.6498      0.137     -4.735      0.000      -0.919      -0.381
Neighborhood_ClearCr    -0.2102      0.144     -1.456      0.146      -0.493       0.073
Neighborhood_CollgCr    -0.0761      0.119     -0.641      0.522      -0.309       0.157
Neighborhood_Crawfor    -0.0823      0.129     -0.638      0.523      -0.335       0.171
Neighborhood_Edwards    -0.7613      0.124     -6.143      0.000      -1.004      -0.518
Neighborhood_Gilbert    -0.0980      0.126     -0.777    

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [31]:
# Your code here
linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=5)
selector = selector.fit(preprocessed.drop('SalePrice_log', axis=1), preprocessed['SalePrice_log'])

#this tells you which variables are selected
display('support variables:',selector.support_ )
#and the ranking
display('support ranking:',selector.ranking_)
#display(pd.DataFrame(selector.support_))
display(list(preprocessed.drop('SalePrice_log', axis=1).columns))
display(list(selector.support_))
#cols = np.dot(list(selector.support_), list(preprocessed.drop('SalePrice_log', axis=1).columns))
K = list(preprocessed.drop('SalePrice_log', axis=1).columns)
L = list(selector.support_)
cols = [k * l for k,l in zip(K, L)]
finalcols = []
for e in cols:
    if e != '':
        finalcols.append(e)
cols = finalcols
display('cols',(cols))
#you can get access to the parameter estimates
estimators = selector.estimator_
print(estimators.coef_)
print(estimators.intercept_)



'support variables:'

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False])

'support ranking:'

array([33, 26, 15, 30, 16, 34, 38,  2,  4,  3, 23, 19, 24, 40, 42, 22, 31,
       25,  1,  1,  1,  1, 13, 39, 10, 17, 32, 37, 36,  8, 35, 11,  7, 29,
       21, 41, 27,  1,  6, 12, 14, 20, 28, 18,  5, 43,  9])

['LotArea_log',
 'FirstFlrSF_log',
 'GrLivArea_log',
 'BldgType_2fmCon',
 'BldgType_Duplex',
 'BldgType_Twnhs',
 'BldgType_TwnhsE',
 'KitchenQual_Fa',
 'KitchenQual_Gd',
 'KitchenQual_TA',
 'SaleType_CWD',
 'SaleType_Con',
 'SaleType_ConLD',
 'SaleType_ConLI',
 'SaleType_ConLw',
 'SaleType_New',
 'SaleType_Oth',
 'SaleType_WD',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Pave',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_Gilbert',
 'Neighborhood_IDOTRR',
 'Neighborhood_MeadowV',
 'Neighborhood_Mitchel',
 'Neighborhood_NAmes',
 'Neighborhood_NPkVill',
 'Neighborhood_NWAmes',
 'Neighborhood_NoRidge',
 'Neighborhood_NridgHt',
 'Neighborhood_OldTown',
 'Neighborhood_SWISU',
 'Neighborhood_Sawyer',
 'Neighborhood_SawyerW',
 'Neighborhood_Somerst',
 'Neighborhood_StoneBr',
 'Neighborhood_Timber',
 'Neighborhood_Veenker']

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

<ipython-input-31-71910e9974e5>:16: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  cols = [k * l for k,l in zip(K, L)]


'cols'

['MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Neighborhood_NoRidge']

[2.82476294 1.58111828 2.3678172  1.43855535 1.53187952]
-2.2675871278278215


Fit the linear regression model again using the 5 selected columns

In [33]:
# Your code here
outcome = 'SalePrice_log'
predictors = preprocessed[cols]
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

#model fit
model = ols(formula=formula, data=preprocessed).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.237
Method:                 Least Squares   F-statistic:                     91.55
Date:                Wed, 11 Aug 2021   Prob (F-statistic):           6.73e-84
Time:                        02:50:05   Log-Likelihood:                -1871.4
No. Observations:                1460   AIC:                             3755.
Df Residuals:                    1454   BIC:                             3786.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -2.2676      0.276     -8.208      0.000      -2.809      -1.726
MSZoning_FV              2.8248      0.297      9.519      0.000       2.243       3.407
MSZoning_RH              1.5811      0.352      4.490      0.000       0.890       2.272
MSZoning_RL              2.3678      0.277      8.533      0.000       1.823       2.912
MSZoning_RM              1.4386      0.283      5.092      0.000       0.884       1.993
Neighborhood_NoRidge     1.5319      0.139     11.026      0.000       1.259       1.804
==============================================================================
Omnibus:                       47.664   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               79.387
Skew:                           0.272   Prob(JB):                     5.77e-18
Kurtosis:                       4.004   Cond. No.                         35.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [40]:
# Your code here
y_hat = selector.predict(preprocessed.drop('SalePrice_log', axis=1))
y_hat

array([0.10023007, 0.10023007, 0.10023007, ..., 0.10023007, 0.10023007,
       0.10023007])

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [54]:
# Your code here
import numpy as np

def sq_err(y_real, y_predicted):
    """
    input
    y_real : true y values
    y_predicted : regression line

    
    return
    squared error between regression and true line (ss_tot)
    """
    if len(y_real) != len(y_predicted):
        return "Error"
    else:
        sq_e = 0
        for r, p in zip(y_real, y_predicted):
            sq_e += (r-p)**2
    return sq_e


# Calculate Y_mean , squared error for regression and mean line , and calculate r-squared

def r_squared(y_real, y_predicted):
    """
    input
    y_real: real values
    y_predicted: regression values
    
    return
    r_squared value
    """
    if len(y_real) != len(y_predicted):
        return "Error"
    else:
        y_bar = y_real.mean()
        y_mean = np.zeros(len(y_real))
        y_mean += y_bar
        
        ssr = sq_err(y_real, y_predicted)
        sst = sq_err(y_real, y_mean)
        #print(sst, y_real.var()*len(y_real))
        r_squared = 1-ssr/sst
        return r_squared

def r_squared_adjusted(y_real, y_predicted, p):
    """
    input
    y_real: real values
    y_predicted: regression values
    p: number of independent variables in the model
    
    return
    r_squared_adjusted value
    """
    if len(y_real) != len(y_predicted):
        return "Error"
    else:
        rs = r_squared(y_real, y_predicted)
        n = len(y_real)
        r_squared_adj = 1-(1-rs)*(n-1)/(n-p-1)
        return r_squared_adj
    

y = preprocessed['SalePrice_log']
y_bar = np.mean(y)
rs = r_squared(y, y_hat)
print('rsquared:', rs)
rsa = r_squared_adjusted(y, y_hat, 5)
print('rsquared adjusted:', rsa)
# r_squared is 0.239434  
# adjusted_r_squared is 0.236818

#These r_squared values are pretty lousy, so I'd want the one with the higher value - namely the statsmodels one that used all variables.s

rsquared: 0.23943418177114095
rsquared adjusted: 0.23681875598630986


## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 